In [1]:
import pandas as pd
import numpy as np

In [2]:
data = {
    "user" : [1,1,1,2,2,3,3,3,3,3,4,4,4,5,5,5,5,6,6,7,7,8,8,9,9,10,10,11,11,11,11,11,11,12,12],
    "movie": [1,3,6,3,4,1,2,4,5,6,2,3,5,3,4,5,6,1,5,2,3,4,6,1,3, 2, 3, 1, 2, 3, 4, 5, 6, 2, 5],
    "rating":[1,2,1,4,2,3,5,4,4,3,4,1,3,2,5,4,3,5,2,4,3,4,2,5,4, 2, 3, 4, 1, 5, 2, 2, 4, 3, 5]
}

In [3]:
data = pd.DataFrame(data)

In [4]:
data

,user,movie,rating
0,1,1,1
1,1,3,2
2,1,6,1
3,2,3,4
4,2,4,2
5,3,1,3
6,3,2,5
7,3,4,4
8,3,5,4
9,3,6,3


In [5]:
table = pd.DataFrame(columns = data["user"].unique())

In [6]:
table["Movie"] = sorted(data["movie"].unique())

In [7]:
table = table.set_index("Movie")

In [8]:
for row in range(len(data)):
    table.loc[data.loc[row, "movie"]][data.loc[row, "user"]] = data.loc[row, "rating"]

In [9]:
table = table.T.values
table

array([[1, nan, 2, nan, nan, 1],
       [nan, nan, 4, 2, nan, nan],
       [3, 5, nan, 4, 4, 3],
       [nan, 4, 1, nan, 3, nan],
       [nan, nan, 2, 5, 4, 3],
       [5, nan, nan, nan, 2, nan],
       [nan, 4, 3, nan, nan, nan],
       [nan, nan, nan, 4, nan, 2],
       [5, nan, 4, nan, nan, nan],
       [nan, 2, 3, nan, nan, nan],
       [4, 1, 5, 2, 2, 4],
       [nan, 3, nan, nan, 5, nan]], dtype=object)

In [10]:
means = pd.pivot_table(data, index = "user", values = "rating").values
means

array([[1.33333333],
       [3.        ],
       [3.8       ],
       [2.66666667],
       [3.5       ],
       [3.5       ],
       [3.5       ],
       [3.        ],
       [4.5       ],
       [2.5       ],
       [3.        ],
       [4.        ]])

table -= means

In [11]:
table = pd.DataFrame(table)

In [12]:
table = table.fillna(0).values

In [13]:
def cosine_similiarity(client, table):
    simScore = [0 for _ in range(len(table))]
    client_mag = sum(client**2)**0.5
    for userIndex in range(len(table)):
        user = table[userIndex]        
        simScore[userIndex] = np.dot(user, client)
        mag = sum(user**2)**0.5
        simScore[userIndex] /= mag*client_mag      
    return simScore

In [14]:
cosine_similiarity(table[0], table)

[1.0000000000000002,
 0.7302967433402214,
 0.282842712474619,
 0.16012815380508716,
 0.3888888888888889,
 0.37904902178945177,
 0.48989794855663565,
 0.18257418583505536,
 0.8288497269823398,
 0.6793662204867575,
 0.9045340337332909,
 0.0]

In [15]:
def fillRating(table, estimate, N = 2):
    for r in range(len(table)):
        row = table[r]
        similiarity = cosine_similiarity(row, table)
        similiarity.pop(r)
        neighbours = np.argsort(similiarity)[::-1][:N]
        
        similiarity = np.array(similiarity)[neighbours]
        similiarity/=sum(similiarity)
        ratings = table[neighbours]
        
        e = np.dot(ratings.T, similiarity)  
        indices = np.where(table[r] == 0)
        for i in indices:
            estimate[r][i] = e[i]
        

In [16]:
estimatedRating = np.zeros((len(table), len(table[0])))

In [17]:
fillRating(table, estimatedRating)

In [18]:
estimatedRating

array([[0.        , 1.04366275, 0.        , 1.91267449, 0.        ,
        0.        ],
       [3.0188696 , 0.        , 0.        , 0.        , 0.        ,
        0.4952826 ],
       [0.        , 0.        , 0.49387102, 0.        , 0.        ,
        0.        ],
       [4.45074636, 0.        , 0.        , 1.09850727, 0.        ,
        2.19701455],
       [1.41585446, 4.47195149, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.86146493, 1.2921974 , 2.27707013, 0.        ,
        1.13853507],
       [2.79283455, 0.        , 0.        , 0.        , 1.32429927,
        0.        ],
       [1.25376858, 2.08961429, 1.16415428, 0.        , 4.        ,
        0.        ],
       [0.        , 0.96250881, 0.        , 0.        , 0.        ,
        0.51874559],
       [0.        , 0.        , 0.        , 0.88218227, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [1.29858534, 0

In [19]:
np.round(estimatedRating)

array([[0., 1., 0., 2., 0., 0.],
       [3., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 1., 0., 2.],
       [1., 4., 0., 0., 0., 0.],
       [0., 1., 1., 2., 0., 1.],
       [3., 0., 0., 0., 1., 0.],
       [1., 2., 1., 0., 4., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 2., 0., 1.]])

In [26]:
def recommend(estimatedRating):
    for u in range(len(estimatedRating)):
        print(u, " : ", np.where(estimatedRating[u] >= 3)[0])

In [27]:
recommend(estimatedRating)

0  :  []
1  :  [0]
2  :  []
3  :  [0]
4  :  [1]
5  :  []
6  :  []
7  :  [4]
8  :  []
9  :  []
10  :  []
11  :  []
